Q4 a)

In [24]:
from ortools.linear_solver import pywraplp
import numpy as np
import pandas as pd

In [25]:
df = pd.read_excel("Call Center Req.xlsx")
# set variables
# set xi,j = i hours shift and take rest at j time spot. x =  {x8,8 ; x8,9; x6,6; x6,7; x3}
variables = [[0 for i in range(5)] for j in range(df.shape[0])]
cost = [200,200,162,162,120]

solver = pywraplp.Solver('Q3', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)
objective = solver.Objective()
objective.SetMinimization()

#set variables x_i,j
#x_i,j = variables[i][j] = number of worker starts work in i label time spot 
#and belongs to x[j] type while x = x =  {x8,8 ; x8,9; x6,6; x6,7; x3}
#for example: x_1,2 = number of worker starts work in 1st label time spot (7:30 am) 
#and belongs to x6,6 (take 6 hours shift and take lunch break at 6th time spots)

for i in range(len(variables)):
    for j in range(len(variables[0])):
        variables[i][j] = solver.IntVar(0, solver.infinity(), "x_"+str(i)+","+str(j))
        #set objective function
        #to minimize(cost_total = sum(variables[i][j]*cost[j]))
        cost_total = objective.SetCoefficient(variables[i][j], cost[j])


In [26]:

#set constrains
#for every x[i], we create a list that missing the lunch break.
#for every actual_number[i][j] within the matrix variables,
#we have lower limit of required_number[i][j].
#for every actual_number[i][j] within the matrix variables,
#we set coefficient = 0 during lunch break and coefficient = 1 else. 

list1 = [0,1,2,3,4,5,6,8,9,10,11,12,13,14,15]
list2 = [0,1,2,3,4,5,6,7,9,10,11,12,13,14,15]
list3 = [0,1,2,3,4,6,7,8,9,10,11]
list4 = [0,1,2,3,4,5,7,8,9,10,11]
list5 = [0,1,2,3,4,5]

actual_number = [0 for i in range(len(variables))]
required_number = list(df.iloc[:,2])

for i in range(len(variables)):
    actual_number[i] = solver.Constraint(required_number[i], solver.infinity())
#x_8,8
for i in range(len(variables)):
    for j in list1:
        if i + j <= len(variables) - 1:
            actual_number[i+j].SetCoefficient(variables[i][0], 1)
            
#x_8,9
for i in range(len(variables)):
    for j in list2:
        if i + j <= len(variables) - 1:
            actual_number[i+j].SetCoefficient(variables[i][1], 1)
        
#x_6,6
for i in range(len(variables)):
    for j in list3:
        if i + j <= len(variables) - 1:
            actual_number[i+j].SetCoefficient(variables[i][2], 1)

#x_6,7
for i in range(len(variables)):
    for j in list4:
        if i + j <= len(variables) - 1:
            actual_number[i+j].SetCoefficient(variables[i][3], 1)

#x_3
for i in range(len(variables)):
    for j in list5:
        if i + j <= len(variables) - 1:
            actual_number[i+j].SetCoefficient(variables[i][4], 1)
        
status = solver.Solve()

Q4 b)

In [27]:
#find the optimal solution by solver
if status == solver.OPTIMAL:
    print("Solver reaches the optimal solution with objective function = " + str(objective.Value()))

else:
    print('Solver cannot find optimal solution')
    
x = np.zeros((len(variables),5))
for i in range(len(variables)):
    for j in range(5):
        x[i][j] = variables[i][j].solution_value()
print(x)

Solver reaches the optimal solution with objective function = 21596.0
[[ 3.  0.  2.  0.  0.]
 [ 0.  3.  2.  0.  0.]
 [ 0.  0.  5.  0.  0.]
 [ 0.  0.  6.  1.  0.]
 [ 0.  2.  0.  3.  0.]
 [ 3.  6.  0.  0.  0.]
 [ 0.  2.  8.  0.  1.]
 [ 0.  0.  0.  1. 10.]
 [ 0.  0.  0.  0.  5.]
 [ 4.  1.  0.  0.  0.]
 [ 5.  0.  0.  0.  0.]
 [ 3.  7.  0.  0.  0.]
 [ 0. 10.  0.  0.  0.]
 [ 0.  4.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.]
 [ 5.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  3.  0.]
 [ 0.  0.  2.  1.  0.]
 [ 0.  0.  4.  0. 11.]
 [ 0.  0.  0.  0.  5.]
 [ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.]]
